In [56]:
import random
import numpy as np
import pandas as pd

In [57]:
def calculate_body_fat_percentage(bmi, age, gender):
    gender_factor = 1 if gender == 'Male' else 0
    
    # Deurenberg Equation
    body_fat_percentage = (1.20 * bmi) + (0.23 * age) - (10.8 * gender_factor) - 5.4
    return body_fat_percentage

In [58]:
def calculate_BMR(age, gender, weight_kg, height_m, activity_level):
    # Calculating BMI
    bmi = weight_kg / (height_m ** 2)

    # Calculating Body Fat Percentage
    body_fat_percentage = calculate_body_fat_percentage(bmi, age, gender)

    # Calculating BMR using Harris Benedict Formula
    if gender == 'Male':
        bmr = 10 * weight_kg + 6.25 * (height_m * 100) - 5 * age + 5
    else:
        bmr = 10 * weight_kg + 6.25 * (height_m * 100) - 5 * age - 161

    # Adjusting BMR for body composition
    lbm = weight_kg * (1 - body_fat_percentage / 100)
    lbm_factor = 1 + (max(lbm / weight_kg - 0.5, 0) * 0.1)
    bmr *= lbm_factor

    # Adjusting BMR based on Harris Benedict Formula
    activity_multipliers = {
        'Sedentary': 1.2,
        'Lightly Active': 1.375,
        'Moderately Active': 1.55,
        'Very Active': 1.725,
        'Extra Active': 1.9
    }
    
    # Additional BMR adjustment for older adults
    if age > 50:
        age_adjustment_factor = 1 - ((age - 50) / 100)  # 1% decrease per year over 50
        bmr *= age_adjustment_factor
    
    bmr *= activity_multipliers[activity_level]
    
    # Round BMR to the nearest multiple of 5 or 10
    nearest_five = round(bmr / 5) * 5
    nearest_ten = round(bmr / 10) * 10

    # Choose the closest rounding
    if abs(nearest_five - bmr) < abs(nearest_ten - bmr):
        return nearest_five
    else:
        return nearest_ten

    return bmr

In [59]:
def generate_profile():
    age = random.randint(18, 60)
    
    gender = np.random.choice(["Male", "Female"])

    # Height assigned randomly in feet and inches
    feet = random.randint(4, 6)
    inches = random.randint(0, 11)
    height_cm = round((feet * 30.48) + (inches * 2.54), 1)  # Convert to cm
    # Weight assigned randomly in kg
    weight_kg = round(random.uniform(45, 150))
    # Convert height to meters and round
    height_m = round(height_cm / 100, 2)

    # assigning activity level
    activity_levels = ['Sedentary', 'Lightly Active', 'Moderately Active', 'Very Active', 'Extra Active']
    activity_level = np.random.choice(activity_levels)

    # calculating BMI
    bmi = round(weight_kg / (height_m ** 2), 1)
    
    # assigning HbA1C value
    hbA1c = round(random.uniform(4, 9), 1)  # HbA1c range from 4% to 7%

    # Calculate BMR
    BMR = round(calculate_BMR(age, gender, weight_kg, height_m, activity_level), 2)

    return {
        'Age': age,
        'Gender': gender,
        'Height (m)': height_m,
        'Weight (kg)': weight_kg,
        'BMI': bmi,
        'HbA1c Level': hbA1c,
        'Activity Level': activity_level,
        'BMR': BMR
    }
# Generate profiles
profile = pd.DataFrame([generate_profile()])
profile.to_csv("./Model 02 Dependencies/User Profile.csv", index=False)


In [60]:
import pandas as pd
import joblib

# Load the trained pipeline
base_path = "./Model 01 Dependencies/"
pipeline = joblib.load(f'{base_path}Model 01 Pipeline.pkl')

# Function to preprocess and predict
def preprocess_and_predict(new_data):
    # Predict using the pipeline
    predictions = pipeline.predict(new_data)
    return predictions

# Example of using the prediction function
new_data = profile
predictions = preprocess_and_predict(new_data)

# Printing predictions
print("Predictions:")
print("------------")
print(f"Carbs%: {predictions[0, 0]:.2f}%")
print(f"Fats%: {predictions[0, 1]:.2f}%")
print(f"Proteins%: {predictions[0, 2]:.2f}%")

Predictions:
------------
Carbs%: 51.66%
Fats%: 23.08%
Proteins%: 27.36%
